In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
# simply put osm files into the data-dir (all will get evaluated)
EVAL_DIR = "/home/felix/todo/osm-tmp"
MLP_METHODS = ["kmeans", "merge", "hop"]
# MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 0.5)]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 1.0)]
QUERY_METHODS = ["pch", "pch-pch"]
AREAS = ["saarland"]
print(MLP_METHODS, "with", MLP_LEVELS)

In [ ]:
df = pd.DataFrame()
for area in AREAS:
    for query in QUERY_METHODS:
        for mlp in MLP_METHODS:
            for partitions in MLP_LEVELS:
                tmp = pd.read_json("/home/felix/todo/osm-tmp/" + area + "-" + mlp + "-" + "_".join(map(str, partitions)) + "-" + query + "-time.json")
                tmp["Area"] = area
                tmp["Query"] = query
                tmp["Mlp"] = mlp
                tmp["Mlp_partitions"] = "_".join(map(str, partitions))
                df = df.append(tmp, ignore_index = True)

In [ ]:
fig, ax = plt.subplots()
speedups = list()
for area in AREAS:
    dijkstra = df[(df.Query == "normal") & (df.Area == area)]["time"].mean()
    bidijkstra = df[(df.Query == "bi") & (df.Area == area)]["time"].mean()
    for query in QUERY_METHODS:
        for mlp in MLP_METHODS:
            x = list()
            y = list()
            for partitions in MLP_LEVELS:
                tmp = df[(df.Area == area) & (df.Query == query) & (df.Mlp == mlp) & (df.Mlp_partitions == "_".join(map(str, partitions)))]
                x.append(partitions[0])
                y.append(tmp["time"].mean())
                speedups.append({"Query": query, "MLP": mlp, "_".join(map(str, partitions)): dijkstra / tmp["time"].mean()})
            plt.plot(x, y, marker=plot_get(query), color=plot_get(mlp), label=query.replace("pch-pch", "raw-pch") + "-" + mlp, alpha=0.7)
plt.xlabel("MLP-Partition-Size")
plt.ylabel("Query-time in ns")
plt.legend(loc='upper left')
fig.savefig("pch.pgf", bbox_inches="tight")